# Preprocessing CBN Data

In [ ]:
from cng.utils import *
from utils import *
from h3_utils import *
s3 = s3_client()

duckdb_install_h3()
from minio.error import S3Error


import zipfile
import os
import subprocess
os.chdir('../data/')

import geopandas as gpd
import ibis
from ibis import _
# con = ibis.duckdb.connect('temp',extensions = ["spatial", "h3"])
# set_secrets(con)

import rasterio
from rasterio.features import shapes
from shapely.geometry import shape
import numpy as np

## Counties **

In [ ]:
%%time 
con = ibis.duckdb.connect('counties',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Counties'
name = 'CA_counties'

# unzip(folder = folder, file = '30x30_Counties.zip')
# process_vector(folder = folder, file = f"{name}.shp")
# convert_pmtiles(folder = folder, file = f"{name}.parquet")

convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols= "COUNTY_NAM")

## Climate Zones **

In [ ]:
%%time 
con = ibis.duckdb.connect('climate_zones',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Climate_zones'
name = 'climate_zones_10'
# download(folder = folder, file = 'clusters_10.tif')
# process_raster(s3, folder = folder, file = 'clusters_10.tif', file_name = f"{name}.tif")
convert_h3(con, s3, folder = folder, file = f"{name}_processed.parquet", cols= "id")

## Ecoregions **

In [ ]:
%%time 
con = ibis.duckdb.connect('ecoregion',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Ecoregion'
name = 'ACE_ecoregions'

unzip(folder = folder, file = '30x30_Ecoregions.zip')
process_vector(folder = folder, file = f"{name}.shp")

convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols= 'CA_Ecoregi')

## Habitat

#### 13 class major habitat types **

In [ ]:
# download(folder = 'Habitat', file = 'CWHR13_2022.tif')
# process_raster(s3, folder = 'Habitat', file = 'CWHR13_2022.tif')

In [ ]:
%%time
con = ibis.duckdb.connect('habitat',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Habitat'
name = 'fveg22_1'

# unzip(folder = folder, file = 'fveg221gdb.zip')

# command = [
#     "gdalwarp",
#     "-of", "GTiff",
#     'fveg22_1.gdb',
#     'fveg22_1.tif' 
#     ]

# subprocess.run(command, check=True)
process_raster(s3, folder = folder, file = f"{name}.tif")
upload(folder = folder, file = f'{name}_processed.tif.aux.xml')

convert_h3(con, s3, folder = folder, file = f"{name}_processed.parquet", cols= "id")

#### 60+ class habitat types

## ACE Biodiversity

In [ ]:
%%time 
folder = 'ACE_biodiversity'
name = 'ACE_terrestrial_biodiversity_summary_ds2739'

download(folder = folder, file = 'Terrestrial_Biodiversity_Summary_-_ACE_[ds2739].geojson',
         file_name = f"{name}.geojson")

process_vector(folder = folder, file = f"{name}.geojson")
# convert_pmtiles(folder = folder, file = f"{name}.geojson")

convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols= "OBJECTID")
# gdf = gpd.read_parquet(f"{name}.parquet")


#### ACE BioRank and Rare Rank 

In [ ]:
# Filter data to rank 5.
ACE_rank_files = ['ACE_biorank_statewide','ACE_biorank_ecoregion',
                  'ACE_rarerank_statewide','ACE_rarerank_ecoregion']
             
ACE_rank_cols =  ['BioRankSW','BioRankEco','RarRankSW','RarRankEco']   

for col,name in zip(ACE_rank_cols,ACE_rank_files):
    cols = ['OBJECTID', 'Hex_ID', 'Eco_Sect', 'Eco_Name',
            'County', 'Shape__Area', 'Shape__Length', 'geometry']
    cols.append(col) #select only the cols we want + the new col. 
    rank_df = gdf[gdf[col]==5][cols]# filter ranks = 5
    process_vector(folder = 'ACE_biodiversity/'+name, file = name+'.parquet',gdf = rank_df)
    convert_pmtiles(folder ='ACE_biodiversity/'+name, file = name+'.parquet')


#### Other ACE Biodiversity **

In [ ]:
ACE_files = ['ACE_amphibian_richness','ACE_reptile_richness',
            'ACE_bird_richness','ACE_mammal_richness',
            'ACE_rare_amphibian_richness','ACE_rare_reptile_richness',
            'ACE_rare_bird_richness','ACE_rare_mammal_richness',
            'ACE_endemic_amphibian_richness','ACE_endemic_reptile_richness',
            'ACE_endemic_bird_richness','ACE_endemic_mammal_richness']

ACE_cols = ['NtvAmph','NtvRept','NtvBird','NtvMamm','RarAmph','RarRept','RarBird','RarMamm',
            'AmphEndem','ReptEndem','BirdEndem','MammEndem']

for col,name in zip(ACE_cols,ACE_files):
    cols = ['OBJECTID', 'Hex_ID', 'Eco_Sect', 'Eco_Name',
            'County', 'Shape__Area', 'Shape__Length', 'geometry']
    cols.append(col) #select only the cols we want + the new col. 
    if name in ['NtvAmph','NtvRept','NtvBird','NtvMamm']:
        percentile = 0.8
    else: 
        percentile = 0.95
    threshold = gdf[col].quantile(percentile)
    ace = gdf[gdf[col]>=threshold][cols]
    process_vector(folder = 'ACE_biodiversity/'+name, file = name+'.parquet',gdf = ace)
    convert_pmtiles(folder ='ACE_biodiversity/'+name, file = name+'.parquet')


# calculate 80% percentile, filter to those >= threshold. 
# subset to calculate acres within each network, % of feature conserved and % of network 

## Biodiversity

#### Plant richness **

In [ ]:
%%time 
con = ibis.duckdb.connect('plant',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Biodiversity_unique/Plant_richness'
name = 'species_D'

download(folder = folder, file = f"{name}.tif")
filter_raster(folder = folder, file = f"{name}.tif", percentile = 80)
convert_h3(con, s3, folder = folder, file = f"{name}_processed.parquet", cols= "id")

#### Rarity-weighted endemic plant richness **

In [ ]:
%%time 
con = ibis.duckdb.connect('end_plant',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Biodiversity_unique/Rarityweighted_endemic_plant_richness'
name = 'endemicspecies_E'

download(folder = folder, file = f"{name}.tif")
filter_raster(folder = folder, file = f"{name}.tif", percentile = 80)
convert_h3(con, s3, folder = folder, file = f"{name}_processed.parquet", cols= "id")

#### Abundance for 26 bird species

## Connectivity and Resilience
#### Resilient Connected Network - all categories **

In [ ]:
%%time 
con = ibis.duckdb.connect('CRN',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Connectivity_resilience/Resilient_connected_network_allcategories'
name = 'rcn_wIntactBioCat_caOnly_2020-10-27'

process_raster(s3, folder = folder, file = f"{name}.tif")
convert_h3(con, s3, folder = folder, file = f"{name}_processed.parquet", cols= "id")

#### Present day connectivity - all categories

#### Climate migration routes

## Freshwater Resources

#### Freshwater species richness

#### Wetlands **

In [ ]:
%%time 

folder = 'Freshwater_resources/Wetlands'
name = 'CA_wetlands'

# only pick a subset 
unzip(folder = folder, file = 'CA_geodatabase_wetlands.zip')
gdf = gpd.read_file('CA_geodatabase_wetlands.gdb')
wetlands = ['Freshwater Emergent Wetland', 'Freshwater Forested/Shrub Wetland', 'Estuarine and Marine Wetland']
gdf = gdf[gdf['WETLAND_TYPE'].isin(wetlands)]

process_vector(folder = folder, file = f"{name}.parquet", gdf = gdf)
# convert_pmtiles(folder =folder, file = f"{name}.parquet")
geom_to_h3(con, folder = folder, file = f"{name}.parquet", cols= ['ATTRIBUTE','WETLAND_TYPE','NWI_ID'])



#### Groundwater dependent ecosystems

#### Streams by order

#### Perennial streams

#### Fish passage barriers

## NBS and Agriculture

#### Drinking water source watersheds

#### Farmland + Land suitable for grazing **

In [ ]:
%%time 

folder = 'NBS_agriculture/Farmland'
unzip(folder = folder, file = 'Important_Farmland_2018.zip')

folder = 'NBS_agriculture/Farmland_all'
name = 'Important_Farmland_2018'
process_vector(folder = folder, file = f"{name}.gdb")
# convert_pmtiles(folder = folder, file =f"{name}.parquet")
convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols= ['county_nam','polygon_ty'])

# only pick a subset 
folder = 'NBS_agriculture/Farmland_all/Farmland'
name = 'Farmland_2018'
# gdf = gpd.read_file('Important_Farmland_2018.gdb')
# farmland_type = ['P','S','L','U'] # prime, statewide importance, local importance, unique
# gdf_farmland = gdf[gdf['polygon_ty'].isin(farmland_type)]
# process_vector(folder = folder, file = f"{name}.parquet", gdf = gdf_farmland)
# convert_pmtiles(folder = folder, file =f"{name}.parquet")



# grazing lands 
folder = 'NBS_agriculture/Farmland_all/Lands_suitable_grazing'
name = 'Grazing_land_2018'

# gdf_grazing = gdf[gdf['polygon_ty'] == 'G']
# process_vector(folder = folder, file = f"{name}.parquet", gdf = gdf_grazing)
# convert_pmtiles(folder = folder, file =f"{name}.parquet")


#### Carbon storage **

## Climate Risks

#### Fire perimeters **

Only YEAR >= 2014. 

In [ ]:
%%time 
folder = 'Climate_risks/Historical_fire_perimeters'
name = 'calfire_2023'

unzip(folder = folder, file = 'fire23-1gdb.zip')
gdf = gpd.read_file('fire23_1.gdb')
gdf = gdf[~gdf['YEAR_'].isna()]
gdf['YEAR_'] = gdf['YEAR_'].astype('int64')
# gdf = gdf[gdf['YEAR_']>=2014]
process_vector(folder = folder, file = f"{name}.parquet", gdf = gdf)
# convert_pmtiles(folder = folder, file = f"{name}.parquet")

convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols= ['INC_NUM','FIRE_NAME','YEAR_'])

#### Flood hazard zones **

#### Sea level rise

#### Mid-century habitat climate exposure **

In [ ]:
'''
First mask out non-natural lands.
A binary natural vs. non-natural  land mask is included in the data package. 
Use the combined group of all values < 0 and >=0.95 as exposed.  
Do seperately for both climate models - CNRM and MIROC.
'''

unzip(folder = 'Climate_risks/Mid-century_habitat_climate_exposure', file = 'Midcentury_habitat_climate_exposure.zip')

# still need to do 

## Progress data - newly protected

#### Newly counted

In [ ]:
%%time 
folder = 'Progress_data_new_protection/Newly_counted_lands'
name = 'newly_counted_lands_2024'

unzip(folder = folder, file = f"{name}.shp.zip")
process_vector(folder = folder, file = f"{name}.shp")
# convert_pmtiles(folder = folder, file = f"{name}.parquet")
convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols= ['ORIG_FID'])


#### DAC **

In [ ]:
%%time 
folder = 'Progress_data_new_protection/DAC'
name = 'DAC_2022'

unzip(folder = folder, file = 'sb535dacgdbf2022gdb.zip')
process_vector(folder = folder, file = 'SB535DACgdb_F_2022.gdb', file_name = f"{name}.parquet")
# convert_pmtiles(folder = folder, file = f"{name}.parquet")
convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols= ['GEOID'])


#### Priority populations

In [ ]:
%%time 
con = ibis.duckdb.connect('priority_pop',extensions = ["spatial", "h3"])
set_secrets(con)

folder = 'Progress_data_new_protection/Priority_populations'
name = 'CalEnviroScreen4'
unzip(folder = folder, file = 'Priority Populations 4.0 Geodatabase.zip')

gdf = (con.read_geo('Priority Populations 4.0 Combined Layer.gdb')
       .mutate(id=ibis.row_number().over()) #making a unique id 
      ).execute().set_crs('EPSG:3857')

process_vector(folder = folder, file = 'Priority Populations 4.0 Combined Layer.gdb',
               file_name = f"{name}.parquet", gdf = gdf)

# convert_pmtiles(folder = folder, file = f"{name}.parquet")
convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols= ["id"])


In [ ]:
file = f"{name}.parquet"
folder = 'Progress_data_new_protection/Priority_populations'
name = 'CalEnviroScreen4'
bucket, path = info(folder, file)
# path, file = os.path.split(path)
# name, ext = os.path.splitext(file)
# join_chunked(bucket, path, file)
con.read_parquet(f"s3://{bucket}/{folder}/hex/{file}_part_000.parquet").head(10).execute()

#### Low income communities **

In [ ]:
folder = 'Progress_data_new_protection/Low_income_communities'
name = 'low_income_CalEnviroScreen4'

unzip(folder = folder, file = 'Priority Populations 4.0 Geodatabase.zip')

gdf = gpd.read_file('Priority Populations 4.0 Combined Layer.gdb')
gdf = gdf[gdf['Designatio'] =='Low-income community']
process_vector(folder = folder, file = f"{name}.parquet", gdf = gdf)
convert_pmtiles(folder = folder, file = f"{name}.parquet")

## Base layer for denominator 

In [ ]:
folder = 'Progress_data_new_protection/Land_Status_Zone_Ecoregion_Counties'
name = 'all_regions_reGAP_county_eco'

unzip(folder = folder, file = 'Land_Status_Zone_Ecoregion_Counties.shp.zip')
process_vector(folder = folder, file = 'Land_Status_Zone_Ecoregion_Counties.shp',
               file_name = f"{name}.parquet")
convert_pmtiles(folder = folder, file = f"{name}.parquet")

# CA Nature data

In [ ]:
# def convert_h3_2(con, folder, file, cols, zoom = "8"):
#     cols = ", ".join(cols) if isinstance(cols,list) else cols #unpack columns  
#     bucket = 'public-ca30x30'
#     name = 'ca-30x30-base'
#     file = 'ca-30x30-base.parquet'
#     folder = "Preprocessing"
#     name= name.replace('-','')


#     # reproject 
#     # (con.read_parquet(f"s3://{bucket}/{file}")
#     #  .mutate(geom = _.geom.convert('epsg:3310','epsg:4326'))
#     # ).to_parquet(f"s3://{bucket}/hex/{file}")

#     con.read_parquet(f"s3://{bucket}/{folder}/{file}", table_name = name)

#     con.sql(f'''
#       WITH t2 AS (
#         WITH t1 AS (
#           SELECT {cols},  ST_Dump(geom) AS geom 
#           FROM {name}
#         ) 
#         SELECT {cols},
#               h3_polygon_wkt_to_cells_string(UNNEST(geom).geom, {zoom}) AS h{zoom}
#         FROM t1
#       )
#       SELECT *, UNNEST(h{zoom}) AS h{zoom} FROM t2
#       ''').to_parquet(f"s3://{bucket}/{folder}/hex/{file}")

In [ ]:
%%time 
con = ibis.duckdb.connect('ca_30x30_base',extensions = ["spatial", "h3"])
set_secrets(con)

# file = 'ca-30x30-base.parquet'
folder = "Preprocessing"
name = 'ca-30x30-base'
# download(folder = folder, file = f"{name}.parquet")

# gdf = gpd.read_parquet(f"{name}.parquet")
process_vector(folder = folder, file = f"{name}.parquet")
convert_h3(con, s3, folder = folder, file = f"{name}.parquet", cols= ["id"])

# convert_h3_2(con, folder = folder, file = f"{name}.parquet", cols= ["id"])



In [ ]:
# url = f"s3://public-ca30x30/{folder}/hex/{name}.parquet"
# # url = f"s3://public-ca30x30/{folder}/{name}.parquet"

# con.read_parquet(url).head(10).execute()


In [ ]:
# file = 'ca-30x30-base.parquet'
# folder = "Preprocessing"
# bucket = 'public-ca30x30'
# con.read_parquet(f"s3://{bucket}/{folder}/hex/{file}").head(10).execute()